# Tidy Data / Dados Organizados

R segue um conjunto de convenções que tornam um layout de dados tabulares muito mais fácil de trabalhar do que outros. Seus dados serão mais fáceis de trabalhar no R se seguirem três regras:

* **Cada variável** no conjunto de dados é colocada em sua **própria coluna**;
* **Cada observação** é colocada em sua **própria linha**;
* **Cada valor** é colocado em sua **própria célula**.
<br>
<br>
<left><img src="./image/tidy_data.png" alt="Drawing" style="width:400px;"/></left>  
Os dados que satisfazem essas regras são conhecidos como **tidy data** ou **dados organizados**.

# Aqui lidaremos com conversão entre os formatos WIDE e LONG


## Importando dados em planilha Excel

A planilha **anorexigenos.xls** contém dados de um ensaio clínico, em que se comparou dois anorexígenos, A e B, e as perdas de peso foram registradas. Deseja-se testar se a diferença observada nas duas amostras é estatisticamente significante, assumindo-se $\alpha = 1\%$.
<br>
<br>
<br>
<br>
<br>
<center><img src="./image/anorexigenos.png" alt="Drawing" style="width:400px;"/></center>
<br>

Esse conjunto de dados estão codificados num arquivo Excel, **anorexigenos.xls**, em duas planilhas, **Grupo_A** e **Grupo_B**, na seguinte forma:
<center><img src="./image/planilha_grupo_A.png" alt="Drawing" style="width:200px;"/></center>
<center><img src="./image/planilha_grupo_B.png" alt="Drawing" style="width:200px;"/></center>

Para podermos analisar esses dados no R é necessário rearrumá-los no formato **long**:
<br>
<br>
<center><img src="./image/planilha_total_long.png" alt="Drawing" style="width:200px;"/></center>

In [ ]:
if (!require("xlsx")) install.packages("xlsx", repo = "https://vps.fmvz.usp.br/CRAN/", 
    dep = TRUE)

library(xlsx)

Leia a primeira planilha Grupo_A do arquivo anorexigenos.xls.  
Primeira linha contém os nomes das variábeis

In [ ]:
mydata_A <- read.xlsx("anorexigenos.xls", sheetName="Grupo_A")

In [ ]:
head(mydata_A)

In [ ]:
mydata_B <- read.xlsx("anorexigenos.xls",sheetName="Grupo_B")
head(mydata_B)

### Crie uma coluna Grupo em cada um dos dataframes:

In [ ]:
mydata_B$Grupo <- "B"

mydata_A$Grupo <- "A"

### Empilhando dois dataframes com rbind

In [ ]:
mydata_total <- rbind(mydata_A, mydata_B)

In [ ]:
print(mydata_total)

In [ ]:
# ?read.xlsx2

In [ ]:
if (!require("readxl")) install.packages("readxl", repo = "https://vps.fmvz.usp.br/CRAN/", 
    dep = TRUE)

library(readxl)

In [ ]:
df_A <- read_excel("anorexigenos.xls",sheet="Grupo_A")
df_A

In [ ]:
df_B <- read_excel("anorexigenos.xls",sheet="Grupo_B")
df_B

## Conversão entre formatos WIDE e LONG

Na prática de análises estatísticas é necessário converter dados entre os formatos "wide" e "long".

Muitas funções do R requerem que os dados estejam no formato "long" em vez de formato "wide", usado frequentemente em programas de análise estatística comerciais tais como SPSS, Minitab e etc.

 O roteiro que segue foi adaptado de [Converting data between wide and long format](http://www.cookbook-r.com/Manipulating_data/Converting_data_between_wide_and_long_format/).
 
No exemplo abaixo, os dois dataframes contêm os mesmos conjuntos de dados em formatos "wide" e "long". Aqui mostraremos como converter de um formato ao outro e vice-versa.

In [ ]:
dadosinicial_wide <- read.table(header=TRUE, text='
sujeito sexo controle cond1 cond2
      1    M      7.9  12.3  10.7
      2    F      6.3  10.6  11.1
      3    F      9.5  13.1  13.8
      4    M     11.5  13.4  12.9
')
# Make sure the subject column is a factor
dadosinicial_wide$sujeito <- factor(dadosinicial_wide$sujeito)

In [ ]:
dadosinicial_long <- read.table(header=TRUE, text='
sujeito sexo condicao medida
      1    M controle    7.9
      1    M    cond1   12.3
      1    M    cond2   10.7
      2    F controle    6.3
      2    F    cond1   10.6
      2    F    cond2   11.1
      3    F controle    9.5
      3    F    cond1   13.1
      3    F    cond2   13.8
      4    M controle   11.5
      4    M    cond1   13.4
      4    M    cond2   12.9
')
# Make sure the subject column is a factor
dadosinicial_long$sujeito <- factor(dadosinicial_long$sujeito)

In [ ]:
dadosinicial_wide

In [ ]:
dadosinicial_long

## Conversão de WIDE --> LONG

### Usando a rotina `gather`

In [ ]:
if (!require("tidyr")) install.packages("tidyr",repo="https://vps.fmvz.usp.br/CRAN/",dep=TRUE)

library(tidyr)

# The arguments to gather():
# - data: Data object
# - key: Name of new key column (made from names of data columns)
# - value: Name of new value column
# - ...: Names of source columns that contain values
# - factor_key: Treat the new key column as a factor (instead of character vector)
dadosinicial_wide

In [ ]:
dados_long <- gather(dadosinicial_wide, condicao, medida, controle:cond2, factor_key=TRUE)
dados_long

Neste exemplo, as colunas que serão juntadas (gathered) são especificados por **controle:cond2**, i.e. todas as colunas posiciondas entre **controle** e **cond2**.  

Outra maneira de se executar a mesma coisa, é listando todas as colunas individualmente, via **gather**:

In [ ]:
gather(dadosinicial_wide, condicao, medida, controle, cond1, cond2)

Se tens intenção de usar esta função, gather( ), dentro de um programa e talvez queiras colocar variáveis contendo os nomes das colunas como parâmetro da função. Neste caso, será necessário usar a função gather_( ), que aceita "strings" em vez de nomes de colunas sem aspas ou apóstrofes.

In [ ]:
keycol <- "condicao"  # coluna chave
valuecol <- "medida"  # variável dependente (VD)
gathercols <- c("controle", "cond1", "cond2") 

gather_(dadosinicial_wide, keycol, valuecol, gathercols)

É possível ou desejável opcionalmente renomear os níveis de fator da coluna de variável, e classificá-los.

In [ ]:
# Renomear nomes de fator de "cond1" and "cond2" to "nivel1" and "nivel2"
levels(dados_long$condicao)[levels(dados_long$condicao)=="cond1"] <- "nivel1"
levels(dados_long$condicao)[levels(dados_long$condicao)=="cond2"] <- "nivel2"

# Sortear por sujeito, depois por condicao
dados_long <- dados_long[order(dados_long$sujeito, dados_long$condicao) , ]
dados_long
#

In [ ]:
levels(dados_long$sujeito)

In [ ]:
# ?order

## Conversão de LONG --> WIDE

### Usando rotina spread da biblioteca `tidyr`

In [ ]:
library(tidyr)

# os argumentos para spread(data, key, value):
# - data: Objeto de dados 
# - key: Nome de coluna contendo os novos nomes de colunas
# - value: Nome da coluna que contém os valores (VD)
dados_wide <- spread(dadosinicial_long, condicao, medida)
dados_wide
#>   sujeito sexo cond1 cond2 controle
#> 1       1    M  12.3  10.7      7.9
#> 2       2    F  10.6  11.1      6.3
#> 3       3    F  13.1  13.8      9.5
#> 4       4    M  13.4  12.9     11.5

À semelhança do **formato long** em que se mudou os nomes do fator, aqui no **formato wide** também se pode mudar os nomes das variáveis coluna:  

In [ ]:
# Renomear de cond1 para nivel1, e cond2 para nivel2
names(dados_wide)[names(dados_wide)=="cond1"] <- "nivel1"
names(dados_wide)[names(dados_wide)=="cond2"] <- "nivel2"

# Aqui podemos reordenar as colunas
dados_wide <- dados_wide[, c(1,2,5,3,4)]
dados_wide
#>   sujeito sexo controle nivel1 nivel2
#> 1       1    M      7.9   12.3   10.7
#> 2       2    F      6.3   10.6   11.1
#> 3       3    F      9.5   13.1   13.8
#> 4       4    M     11.5   13.4   12.9

A ordem dos níveis dos fatores determina a ordem de aparecimento das colunas. A ordem dos níveis pode ser alterado antes de "reshaping", ou as colunas poder ser reordenados depois.

## Biblioteca reshape2

### Conversão  WIDE => LONG

Uso `melt`:

In [ ]:
if (!require("reshape2")) install.packages("reshape2", repo = "https://vps.fmvz.usp.br/CRAN/", 
    dep = TRUE)

library(reshape2)

In [ ]:
?melt

In [ ]:
dadosinicial_wide
#>   sujeito sexo controle cond1 cond2
#> 1       1    M      7.9  12.3  10.7
#> 2       2    F      6.3  10.6  11.1
#> 3       3    F      9.5  13.1  13.8
#> 4       4    M     11.5  13.4  12.9



# Especificar id.vars: as variáveis para manter mas não separá-los
melt(dadosinicial_wide, id.vars=c("sujeito", "sexo"))
#>    sujeito sexo  variable value
#> 1        1    M  controle   7.9
#> 2        2    F  controle   6.3
#> 3        3    F  controle   9.5
#> 4        4    M  controle  11.5
#> 5        1    M     cond1  12.3
#> 6        2    F     cond1  10.6
#> 7        3    F     cond1  13.1
#> 8        4    M     cond1  13.4
#> 9        1    M     cond2  10.7
#> 10       2    F     cond2  11.1
#> 11       3    F     cond2  13.8
#> 12       4    M     cond2  12.9

Note que os nomes para duas colunas à direita foram atribuídas automaticamente pela rotina melt, "variable" e "value".

Há opções de `melt` que torna a saída um pouco mais adequada para o trabalho:

In [ ]:
dados_long <- melt(dadosinicial_wide,
        # ID variables - todas as variáveis a preservar mas não separadas.
    id.vars=c("sujeito", "sexo"),
        # Fontes das colunas
    measure.vars=c("controle", "cond1", "cond2" ),
        # Nome de coluna destino que identificará a coluna original
        # de onde a medida proveio
    variable.name="condicao",
    value.name="medida"
)

In [ ]:
dados_long
#>    sujeito sexo   condicao  medida
#> 1        1    M   controle     7.9
#> 2        2    F   controle     6.3
#> 3        3    F   controle     9.5
#> 4        4    M   controle    11.5
#> 5        1    M      cond1    12.3
#> 6        2    F      cond1    10.6
#> 7        3    F      cond1    13.1
#> 8        4    M      cond1    13.4
#> 9        1    M      cond2    10.7
#> 10       2    F      cond2    11.1
#> 11       3    F      cond2    13.8
#> 12       4    M      cond2    12.9

Se omitir o argumento `measure.vars`, `melt( )` usará automaticamente todas as outras variáveis como `id.vars`. O complemento é verdadeiro se `id.vars` for omitido.

Como visto, se não se especificar `variable.name`, `melt()` nomeará a coluna como "variable", e se  `value.name` for omitido, a coluna de VD será denominada "measurement".

Opcionalmente pode-se renomenar os níveis dos fatores da variável coluna.

In [ ]:
# Renomear nomes de fator de "cond1" e "cond2" para, por exemplo, "prima" e "seconda"
levels(dados_long$condicao)[levels(dados_long$condicao)=="cond1"] <- "prima"
levels(dados_long$condicao)[levels(dados_long$condicao)=="cond2"] <- "seconda"

# Ordenar primeiro por sujeito, depois pela condição
dados_long <- dados_long[ order(dados_long$sujeito, dados_long$condicao), ]
dados_long
#>    sujeito sexo   condicao medida
#> 1        1    M   controle    7.9
#> 5        1    M      prima   12.3
#> 9        1    M    seconda   10.7
#> 2        2    F   controle    6.3
#> 6        2    F      prima   10.6
#> 10       2    F    seconda   11.1
#> 3        3    F   controle    9.5
#> 7        3    F      prima   13.1
#> 11       3    F    seconda   13.8
#> 4        4    M   controle   11.5
#> 8        4    M      prima   13.4
#> 12       4    M    seconda   12.9

### Conversão LONG => WIDE
O seguinte código usa `dcast` para remodelar (reshape) os dados. Esta função é destinada para **data frames**; se estiver lidando com **arrays ou matrizes**, use `acast`.

In [ ]:
library(reshape2)

In [ ]:
dadosinicial_long
#>    sujeito sexo   condicao  medida
#> 1        1    M   controle     7.9
#> 2        2    F   controle     6.3
#> 3        3    F   controle     9.5
#> 4        4    M   controle    11.5
#> 5        1    M      cond1    12.3
#> 6        2    F      cond1    10.6
#> 7        3    F      cond1    13.1
#> 8        4    M      cond1    13.4
#> 9        1    M      cond2    10.7
#> 10       2    F      cond2    11.1
#> 11       3    F      cond2    13.8
#> 12       4    M      cond2    12.9

In [ ]:
# Da fonte:
# "sujeito" e "sexo" são colunas que desejmos manter na mesma forma
# "condicao" é a coluna que contém os nomes das novas colunas destinos para colocar as coisas da 
# "medida" para conter as medidas (VD)
dados_wide <- dcast(dadosinicial_long, sujeito + sexo ~ condicao, value.var="medida")
dados_wide
#>   sujeito sexo controle cond1 cond2
#> 1       1    M      7.9  12.3  10.7
#> 2       2    F      6.3  10.6  11.1
#> 3       3    F      9.5  13.1  13.8
#> 4       4    M     11.5  13.4  12.9

Como sempre, opcionalmente pode-se modificar a nomeação das colunas de forma adequada.

In [ ]:
# Renomear cond1 para prima, e cond2 para seconda
names(dados_wide)[names(dados_wide)=="cond1"] <- "prima"
names(dados_wide)[names(dados_wide)=="cond2"] <- "seconda"

# Reordernar as colunas
dados_wide <- dados_wide[, c(1,2,5,3,4)]
dados_wide
#>   sujeito sexo controle prima seconda
#> 1       1    M      7.9  12.3    10.7
#> 2       2    F      6.3  10.6    11.1
#> 3       3    F      9.5  13.1    13.8
#> 4       4    M     11.5  13.4    12.9